# Lunar Lander 2: Migrating from CMA-ME to CMA-MAE

In the [previous tutorial](https://docs.pyribs.org/en/stable/tutorials/lunar_lander.html), we showed how to implement the CMA-ME algorithm in pyribs. Recent work introduced [CMA-MAE](https://arxiv.org/abs/2205.10752), an algorithm which builds on and improves CMA-ME. CMA-MAE not only has strong theoretical guarantees; it also empirically outperforms CMA-ME in a variety of domains. In this tutorial, we show how to implement this more advanced algorithm in pyribs.